# GEOL 593: Seismology and Earth Structure

## Lab assignment 8: Receiver functions

In this lab we will get experience calculating reciever functions from real data recorded by a seismometer in Illinois in order to infer some properties of the crust and mantle. The dataset we will work with is located in `../data/lab_08`, and consists of 3-component broadband data recorded from teleseismic earthquakes at station N4.N41A near Monmouth in western Illinois.

If you are interested in how the data was initially collected, I used the tool ObspyDMT (https://github.com/kasra-hosseini/obspyDMT), which makes it very easy to download and process bulk data for events that match your criteria. For teleseismic receiver function analysis, we are interested in events between 30 - 90 degrees epicentral distance. Typically, studies will use event magnitudes between ~ 5.5 - 7, although here we used a lower cutoff of 6.3 to limit the size of the dataset. For those curious, the data in this study was downloaded with ObspyDMT using the command line argument

`obspyDMT --datapath N41A_10hz --min_date 2014-01-01 --max_date 2023-01-01 --min_mag 6.3 --max_mag 7.0 --min_epi 30 --max_epi 90 --event_catalog IRIS --net N4 --sta N41A --cha HH* --preset 30 --offset 120 --instrument_correction --data_source IRIS --cut_time_phase --sampling_rate 10.0`

Generally, a fair proportion of the events will not yeild useable signals, so another step of manual quality control is usually necessary. After this step, we were left with data from 92 events, which we will use for receiver function analysis. The seismic data is in MSEED format, and is located in `../data/lab_08/events`. There are 3 MSEED files (1 for each component) in each event sub-directory. The metadata of each event, including its longitude, latitude,depth,  and magnitude is located in the file `../data/lab_08/event_catalog.txt`. Run the code below to read this file, and generate lists of each event attribute. 

In [ ]:
#imports
import obspy
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from obspy.clients.fdsn import Client
from obspy.geodetics import gps2dist_azimuth
from scipy import signal
import cmath

#station info
client = Client("IRIS")
inv = client.get_stations(network='N4',station='N41A',level='response')
st_lon = -90.85
st_lat = 40.70

#read catalog
catalog = open('../data/lab_08/event_catalog.txt','r')
lines = catalog.readlines()
ev_names = []
ev_lats = []
ev_lons = []
ev_deps = []
ev_mags = []
ev_dists = []
for i,line in enumerate(lines):
    
    #skip line 1
    if i == 0:
        continue
    #otherwise, append items to list
    else:   
        items = line.split()
        ev_names.append(items[0])
        ev_lat = float(items[1])
        ev_lon = float(items[2])
        ev_dep = float(items[3])
        ev_mag = float(items[4])
        ev_lats.append(ev_lat)
        ev_lons.append(ev_lon)
        ev_deps.append(ev_dep)
        ev_mags.append(ev_mag)
        
        #calculate distance
        dist_m,az,baz = gps2dist_azimuth(ev_lat,ev_lon,st_lat,st_lon)
        ev_dists.append(dist_m/1000.)
        


###  <font color='red'>Question 1 </font> 

Using cartopy, make a map of the location of the seismic station and all of the events. Given the large range of epicentral distances and azimuths, an appropriate choice for the map projection is "AzimuthalEquidistant" (see list of possible projections here: https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html#cartopy-projections).

Hint: When plotting scatter data on a global map, be sure to use the option `ax.set_global()`. Otherwise, the figure may be re-centered and only show the region of the map where there is data plotted. Also, using `ax.scatter()` requires you to set a `transform` argument. In the past, when making mercator maps, you have used `transform=ccrs.Geodetic()`. When using the AzimuthalEquidistant project, set `transform=ccrs.PlateCarree()` to ensure your data is plotted in the proper location.

In [ ]:
#Answer Q1 here.

### Calculating receiver functions

Calculating radial P-to-S receiver functions requires several steps. First, you must trim the 3-component data in a window surrounding the P-wave. The data in `../data/lab_08/events` is already appropriately windowed, so you don't have to worry about this. All waveforms have been trimmed so that they begin 30 s prior to the expected P-wave arrival, and end 120-s after P. Next, you need to rotate the data into the 'RTZ' coordinate system. In this coordinate system, the P-wave is predominantly on the vertical (Z) component, and the P-to-S conversions are predominantly on the radial (R) component. The last step is to deconvolve the P-wave (approximated by the vertical component seismogram) from the radial component. The resulting time series is the radial receiver function.

The deconvolution step is not trivial. There are multiple ways to perform deconvolution, and some of them are more stable than others. In class, you have seen how deconvolution can be performed as a 'spectral division' (i.e., division of two signals in the frequency domain). To avoid division by zero, this approach needs to be stabilized by a 'water-level' parameter. The water level value is a subjective choice, although there are ways for determining an optimal value. Another approach is the use time-domain deconvolution (e.g., Liggoria & Ammon, 1999), which has some advantages of traditional water-level deconvolution.

Below, the function `calc_rf` is provided to calculate the radial receiver function, provided the radial and vertical component traces. The function also provides a choice of how the deconvolution is performed. You can choose `method='water_level'` to use frequency-domain water-level deconvolution, or `method='iterative_decon'` to use time-domain iterative deconvolution. In the former case, the `alpha` parameter sets the water level value.

In [ ]:
def calc_rf(tr_r,tr_z,method='water_level',alpha=0.1):
    '''
    calculates the radial receiver function by deconvolving the vertical component from the radial component
    
    method: 'water_level' or 'iterative_decon'
    '''
    a = tr_r.data
    b = tr_z.data
    
    if method == 'water_level':
        rf = water_level_decon(a,b,dt=tr_z.stats.delta,wl=alpha)
    elif method == 'iterative_decon':
        rf = iterative_deconvolution(a,b,dt=tr_z.stats.delta)
    
    return rf

def water_level_decon(comp2, comp1, dt, wl=0.001,):
    '''
    Water-level deconvolution: Modified from Sanne Cottaar's smurfpy receiver function code (https://github.com/oxalorg/smurf/blob/master/smurf.py)

    Deconvolves comp2 by comp1
    #--------------------------------------------------------
    comp2 = numerator
    comp1 = denominator
    dt = 1/(sampling rate) seconds
    wl = the water-level, usually ranges from 1e-5 to 1e-1
    '''
    
    #Set some parameters
    fmax=0.5 #cutoff frequency for filter
    timeshift=30. #shift between beginning of RF and first arrival

    # Pad the reference component
    padded = np.zeros_like(comp2)
    padded[0:comp1.size] = comp1

    # Convert to frequency domain
    c2freq = np.fft.fft(comp2)
    c1freq = np.fft.fft(padded)
    frq    = np.fft.fftfreq(len(comp2),dt)

    # Construct the denominator, fill in the troughs to stabalize the deconvolution
    wlfreq = c1freq * c1freq.conjugate()
    wlfreq[wlfreq.real < wl*wlfreq.real.max()] = complex(wl*wlfreq.real.max(),0.) # water-level
    
    filterf = np.square( np.cos( np.pi*frq/(2.*fmax) ) ) # fmax is the cut-off frequency
    filterf[np.abs(frq)>fmax] = 0.

    # Perform deconvolution and filter
    newfreq = c2freq * c1freq.conjugate() / (wlfreq)
    newfreq = newfreq * filterf
    # Phase shift to recover time
    phaseshift = [cmath.exp(1j* (-frq[x]*2.*np.pi*timeshift) ) for x in range(len(frq))]
    newfreq    = newfreq * phaseshift
    # Convert back to time
    RF  = np.fft.ifft((newfreq),len(comp2))

    # Reconvolve with vertical component
    conv=np.real(np.convolve(RF,padded,'full'))
    conv=conv[int(timeshift/dt):int(timeshift/dt+len(comp2))]
    
    # Calculate fit
    comp2f=np.real(np.fft.ifft(filterf*np.fft.fft(comp2)))
    # Calculate residual and fit
    residual=comp2f-conv
    fit= 100.* (1.- sum(residual*residual)/sum(comp2f*comp2f))
    decon = RF.real

    return decon

def iterative_deconvolution(comp2,comp1,dt):
    '''
    Perform iterative time domain deconvoliution (e.g., Ligorria & Ammon 1999)
    Modified from Sanne Cottaar's smurfpy code
    '''
    #Set some parameters
    fmax=0.5 #cutoff frequency for filter
    timeshift=30. #shift between beginning of RF and first arrival
    maxbumps=200 #maximum number of pulses to construct RF
    
    # Frequency domain
    NFFT = len(comp2)
    frq  = np.fft.fftfreq(NFFT,dt)

    #cosine filter
    filterf = np.square( np.cos( np.pi*frq/(2.*fmax) ) ) # fmax is the cut-off frequency
    filterf[np.abs(frq)>fmax] = 0.

    # Filter components
    comp2f=np.real(np.fft.ifft(filterf*np.fft.fft(comp2)))
    comp1f=np.real(np.fft.ifft(filterf*np.fft.fft(comp1)))

    # Computer power in numerator
    power=sum(comp1f*comp1f)
    # Initialize values
    maxind=np.empty([maxbumps,1],dtype=int)
    maxvalue=np.empty([maxbumps,1],dtype=float)
    residual=comp2f # Initial residual is horizontal component
    fit_old=0 # Initial fit = 0%

    for peak in range(maxbumps):
    # Correlate signals and find peak
        corr=(np.correlate(residual,comp1f,'full'))
        corr=corr[len(comp2)-int(timeshift/dt):-int(timeshift/dt)]# Set maximum Xcorr to be at the predicted main arrival
        maxind[peak]=np.argmax(np.absolute(corr))# Peak index
        maxvalue[peak]=corr[maxind[peak]]/(power)# Peak amplitude

        # Build deconvolution result
        decon=np.zeros_like(comp2)
        for ind in range(peak+1):
            decon[maxind[ind]]=decon[maxind[ind]]+maxvalue[ind]

        decon=np.real(np.fft.ifft(filterf*np.fft.fft(decon)))    # Filter deconvolution result

        # Reconvolve with vertical component
        conv=np.real(np.convolve(decon,comp1,'full'))
        conv=conv[int(timeshift/dt):int(timeshift/dt)+len(comp2)]

        # Calculate residual and fit
        residual=comp2f-conv
        fit= 100.* (1.- sum(residual*residual)/sum(comp2f*comp2f))
        if (fit-fit_old)<0.01:
            break
        fit_old=fit

    return decon

In [ ]:
def calc_rfs_all(ev_lons,ev_lats,ev_names,method='water_level',alpha=0.005):

    data_dir = '/Users/rmaguire/Desktop/geol_593/data/lab_08'
    rf_list = []

    for i in range(0,len(ev_names)):
        #print('calculating RF for event {} ({}/{})'.format(ev_names[i],i+1,len(ev_names)))
    
        #read seismic data
        st = obspy.read('{}/events/{}/*'.format(data_dir,ev_names[i]))
    
        #pre-processing (filter, and rotate)
        st.taper(0.05)
        st.detrend()
        st.filter('bandpass',freqmin=1./5,freqmax=2.0,corners=2,zerophase=False)
        st.rotate('->ZNE',inventory=inv) #rotate INTO ZNE (assuming some channels are BH1, BH2)
        #dist_m,az,baz = gps2dist_azimuth(evla,evlo,stla,stlo) #calculate back-azimuth
        dist_m,az,baz = gps2dist_azimuth(ev_lats[i],ev_lons[i],st_lat,st_lon) #calculate back-azimuth
        st.rotate('NE->RT',back_azimuth=baz) #rotate from ZNE to RTZ
    
        #calculate receiver functions
        tr_z = st.select(channel='*Z')[0]
        tr_r = st.select(channel='*R')[0]
        rf = calc_rf(tr_r.copy(),tr_z.copy(),method=method,alpha = alpha)
        rf_list.append(rf)
        
    rfs = np.array(rf_list)
        
    return rfs

###  <font color='red'>Question 2 </font>

The function above `calc_rfs_all` calculates radial receiver functions for all of the events. This function is loops through each event, performs some pre-processing, and calls the `calc_rf`. Each receiver function is added to a numpy array (named `rfs`) which has the shape of (N,M), where N is the number of receiver functions (should be 92 in this case) and M is the number of points in a single receiver function (should be 750 here, which is the same number of points in each original seismogram). The function requires the parameters `ev_lons`,`ev_lats`,and `ev_names`, which are lists that were calculated above when we read the event catalog. You must also specify the deconvolution method and, optionally the water level parameter alpha. The function returns the numpy array containing all of the receiver functions (each row of the matrix is an individual receiver function).

i) use `calc_rfs_all` to calculate receiver functions for both deconvolution methods. The default water-level parameter is fine, although you can adjust this to see the influence.

ii) Plot the 'stack' (i.e., the average) of all of individual receiver functions for both deconvolution methods (i.e., you will plot one stack of the receiver functions calculated with water-level deconvolition, and one stack of the receiver functions calculated with iterative time domain deconvolution. The x-axis of the plot should be in time (s) and the y-axis should be the receiver function amplitude.In order to correctly plot the time axis, you will need to create a time vector, ranging from -30 to 120, with the same number of points as the receiver function stack.

*Hint, you can calculate the average of a numpy array with `np.mean(rfs,axis=0)`, where `rfs` is the array containing all the receiver functions*

In [ ]:
#Answer Q2 here.

###  <font color='red'>Question 3</font> 

The main peak (at t = 0) corresponds to the P-wave arrival. Subsuquent pulses may indicate either P-to-S conversions below the station, or multiply reflected phases in the crust. In your stack, you should observe a clear upward pulse immediately after the P-wave arrival. This corresponds to the P-to-S conversion at the Moho (called the Pms phase). The second noticeable upward pulse (near 15-s) is the crustal multiple PpPs.

i) What is the travel time of the P-to-S conversion from the Moho, relative to P? In other words, what is the peak-to-peak time difference between the direct P-wave and the Pms conversion? To answer accurately, you may want to zoom in on the plot you made above with the `ax.set_xlim()` function (e.g., `ax.set_xlim([-10,40])`).

ii) Assuming an average Vp and Vs in the crust of 6.3 and 3.5 km/s, respectively, what is the approximate thickness of the crust below this station?

*Hint* The equation for the crustal thickness, assuming the P and P-to-S conversion are traveling vertically through the crust (a reasonably accuarate assumption) is

$
\displaystyle
H = \frac{\Delta t}{\frac{1}{V_S}-\frac{1}{V_P}}
$

where $H$ is the crustal thickness and $\Delta t$ is the travel time delay of the P-to-S conversion at the Moho (i.e., the value you found in part i ).

iii) How does your value of crustal thickness found for this station in western Illinois compare to values found in the literature for this region? For example, compare your crustal thickness estimate to the global model CRUST2.0, or the North American model of Shen & Ritzwoller (2016). (Figure 13 in this paper: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016JB012887)

In [ ]:
#Answer Q3 here

### Transition zone receiver functions

Radial receiver functions are not only useful for imaging the crust-mantle boundary. The have also found extensive application in imaging the seismic discontinuties associated with mineral phase transitions in the mantle transition zone. The two predominant discontinuities in the mantle transition zone occur at approximately 410-km depth (associated with the olivine to wadsleyite mineral phase change) and 660-km depth (associated with the ringwoodite to bridgmanite + ferropericlase transition). These seismically observable discontinuities are often referred to as 'the 410' and 'the 660', respectively, even though the true depths to these boundaries may vary regionally from the nominal depths.

Similarly to the analysis above for imaging the Moho, identifying clear P-to-S conversions from the 410 and the 660 often requires stacking receiver functions for multiple events. However, for these deeper conversions, we need to make an additional consideration. This is because the predicted travel times of deeper P-to-S conversions depend more strongly on the epicentral distance of the event. The dependence of the phase arrival time on the distance is referred to as the 'moveout'. Therefore, before we stack signals from different events, we must account for the moveout. One way to do this is using a 'delay and sum' approach, where individual receiver functions are shifted in time by some amount prior to stacking. The amount which they are shifted is determined by the epicentral distance, and a reference distance (here we use 64 degrees). 

The function below `delay_and_sum` is provided to make a 'moveout corrected stack'. The function takes a numpy array of receiver functions `rfs`, the lists of earthquake depths and distances(`ev_deps` and `ev_dists`), and the name of the phase for which to calculate the moveout (e.g., 'P410s' for the P-to-S conversion at 410-km depth).

In [ ]:
from obspy.taup import TauPyModel
from obspy.geodetics import kilometer2degrees
def delay_and_sum(rfs,ev_deps,ev_dists,phase,ref_dist=64.0):
    '''
    Shifts the signal to account for moveout of a given phase

    inputs---------------------------------------------------------------------
    rfs: array of receiver functions
    ev_deps: list of event depths
    ev_dists: list of event distances (assumes in km)
    phase:   phase name for which to apply moveout correction ("Pms", "P410s", "P660s")
    ref_dist: float, reference epicentral distance

    '''
    model = TauPyModel('PREM')
    rfs_new = np.zeros(rfs.shape)
    for i in range(0,len(ev_lons)):
        t_ref = model.get_travel_times(source_depth_in_km=ev_deps[i],
                                       distance_in_degree=ref_dist,
                                       phase_list=["P",phase])
        t_arr = model.get_travel_times(source_depth_in_km=ev_deps[i],
                                       distance_in_degree=kilometer2degrees(ev_dists[i]),
                                       phase_list=["P",phase])
        for arr in t_ref:
            if arr.name=='P':
                p_arrival_ref = arr.time
            elif arr.name==phase:
                pds_arrival_ref = arr.time

        for arr in t_arr:
            if arr.name=='P':
                p_arrival = arr.time
            elif arr.name==phase:
                pds_arrival = arr.time

        time_shift = (pds_arrival_ref-p_arrival_ref)-(pds_arrival-p_arrival)
        int_shift  = int(time_shift/0.2) #0.2 is 1./samping_rate of this data
        data = rfs[i,:]
        data *= signal.tukey(len(data)) #apply a tapered window (avoid edge artifacts)
        rfs_new[i,:] = np.roll(data,int_shift)
    
    rf_stack = np.mean(rfs_new,axis=0)
    return rf_stack

###  <font color='red'>Question 4</font> 

i)Use the function `delay_and_sum` to calculate moveout corrected receiver function stacks based on 3 different phases: Pms (the moho conversion), P410s (the 410 conversion) and P660s (the 660 conversion). You can use the receiver functions calculated with either the water level or the time domain deconvolution methods. The time domain results tend to be a little cleaner. Using the same axes, plot all three of the stacks. Be sure to use labels and a figure legend.

ii) A P-to-S conversion should maximize in amplitude if the proper moveout correction is applied. For example, if the moveout correction is performed for the P410s phase, the P410s signal should be maximized in the stack, and would be diminished if the moveout correction was applied for a different phase. At approximately what times do you notice signals 'maximizing' when performing the moveout corrections for P410s and P660s? Only one peak (corresponding to the P410s or P660s) should maximize. Focus on signals after ~20 s. We already know that the first two prominent peaks following P are the Pms and PpPs phase.

iii) PREM predicts a P660s - P410s time of 26.3 s. How does this compare to your observations?

In [ ]:
#Answer Q4 here.

###  <font color='red'>Question 5</font> 

If the travel time difference $P660s - P410s$ is smaller than average, what could this imply about the transition zone structure? Think in terms of temperature and/or thickness of the transition zone.

In [ ]:
#Answer Q5 here.